In [ ]:
!pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 22.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

from matplotlib.cm import get_cmap
warnings.filterwarnings("ignore")

In [ ]:
#@title File check
import os
from tqdm.auto import tqdm
directory_path='/content/drive/MyDrive/AP_HW/ERA5 Data/ght_200&500'
length=len(sorted(os.listdir(directory_path)))
print('number of files:',length)
for filename in tqdm(sorted(os.listdir(directory_path))):
  if filename.endswith('.nc'):
    file_path=os.path.join(directory_path,filename)
    ds=xr.open_dataset(file_path)
    year=str(ds.valid_time.dt.year.values[1])
    length=len(sorted(os.listdir(directory_path)))
    file_year=filename[-7:-3]
    if file_year!=year: print(filename)

number of files: 54


  0%|          | 0/54 [00:00<?, ?it/s]

In [ ]:
ds=xr.open_mfdataset('/content/drive/MyDrive/AP_HW/ERA5 Data/ght_200&500/z_200&500-1971.nc')
ds

<xarray.Dataset> Size: 4GB
Dimensions:         (valid_time: 2920, pressure_level: 2, latitude: 281,
                     longitude: 601)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 1971-01-01 ... 1971-12-3...
  * pressure_level  (pressure_level) float64 16B 500.0 200.0
  * latitude        (latitude) float64 2kB 70.0 69.75 69.5 ... 0.5 0.25 0.0
  * longitude       (longitude) float64 5kB 0.0 0.25 0.5 ... 149.5 149.8 150.0
    expver          (valid_time) <U4 47kB dask.array<chunksize=(2920,), meta=np.ndarray>
Data variables:
    z               (valid_time, pressure_level, latitude, longitude) float32 4GB dask.array<chunksize=(730, 1, 71, 151), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-21T06:12 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
# @title Climatology Calculation - MAM
import os
from tqdm.auto import tqdm
import datetime
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))

z_list = []

directory_path = '/content/drive/MyDrive/AP_HW/ERA5 Data/ght_200&500'

# Loop over yearly files
for filename in tqdm(sorted(os.listdir(directory_path))):
  if filename.endswith('.nc'):
    file_path = os.path.join(directory_path, filename)
    year = filename[-7:-3]

    # Open file and select MAM period
    ght = xr.open_dataset(file_path)
    start_time = str(np.datetime64(f'{year}-03-01T00:00:00.000000000') - np.timedelta64(6, 'h'))
    end_time = str(np.datetime64(f'{year}-05-31T17:00:00.000000000'))
    time_range = slice(start_time, end_time)
    ght = ght.sel(valid_time=time_range, pressure_level=500)

    # adjust to Bangladesh timezone
    ght['valid_time'] = ght['valid_time'] + np.timedelta64(6, 'h')

    # Yearly mean
    ds_ght = ght.groupby('valid_time.year').mean(dim='valid_time', skipna=True)
    z_list.append(ds_ght)

# Combine years and compute climatology
z_ds = xr.concat(z_list, dim='year')
z_clim_ds = z_ds.sel(year=slice(1981, 2010))
z_climatology = z_clim_ds.mean(dim='year', skipna=True)

# Save climatology
z_climatology.to_netcdf(f'/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/z/ght_500_mam_climatology.nc')

print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))


2025-06-21 10:22:06.478635


  0%|          | 0/54 [00:00<?, ?it/s]

2025-06-21 10:38:09.863788


In [ ]:
# @title Climatology Calculation - JJAS
import os
from tqdm.auto import tqdm
import datetime
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))  # Print start time

z_list = []

directory_path = '/content/drive/MyDrive/AP_HW/ERA5 Data/ght_200&500'

# Loop over yearly files
for filename in tqdm(sorted(os.listdir(directory_path))):
  if filename.endswith('.nc'):
    file_path = os.path.join(directory_path, filename)
    year = filename[-7:-3]

    # Open file and select JJAS period
    ght = xr.open_dataset(file_path)
    start_time = str(np.datetime64(f'{year}-06-01T00:00:00.000000000') - np.timedelta64(6, 'h'))
    end_time = str(np.datetime64(f'{year}-09-30T17:00:00.000000000'))
    time_range = slice(start_time, end_time)
    ght = ght.sel(valid_time=time_range, pressure_level=500)

    # adjust to Bangladesh timezone
    ght['valid_time'] = ght['valid_time'] + np.timedelta64(6, 'h')

    # Yearly mean
    ds_ght = ght.groupby('valid_time.year').mean(dim='valid_time', skipna=True)
    z_list.append(ds_ght)

# Combine years and compute climatology
z_ds = xr.concat(z_list, dim='year')
z_clim_ds = z_ds.sel(year=slice(1981, 2010))
z_climatology = z_clim_ds.mean(dim='year', skipna=True)

# Save climatology
z_climatology.to_netcdf(f'/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/z/ght_500_jjas_climatology.nc')

print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))  # Print end time


2025-06-21 10:38:09.888088


  0%|          | 0/54 [00:00<?, ?it/s]

2025-06-21 10:56:24.573330
